In [1]:
# Importing libararies
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

In [2]:
#Reading files
num_of_classes = 10
Test_Data = pd.read_csv('Test_Data.csv').values
Test_Labels = pd.read_csv('Test_Labels.csv').values
num_test, col = Test_Data.shape
Test_Data = Test_Data.reshape((num_test, col, 1))
Train = pd.read_csv('Train_Data.csv').values
Train_L = pd.read_csv('Train_Labels.csv').values
M, col = Train.shape
Train = Train.reshape((M, col, 1))
TL = np.zeros((num_of_classes, M, 1))
for i in range(num_of_classes):
    for j in range(M):
        if Train_L[j][0] == i:
            TL[i][j] = 1
        else:
            TL[i][j] = -1
            
Test_L = np.zeros((num_of_classes, num_test, 1))
for i in range(num_of_classes):
    for j in range(num_test):
        if Test_Labels[j][0] == i:
            Test_L[i][j] = 1
        else:
            Test_L[i][j] = -1

In [4]:
def steepest_descent(Train, TL, Test_Data, Test_L, Test_Labels, alpha, landa, dec):
    num_test, col, _ = Test_Data.shape
    num_of_classes, M, _ = TL.shape
    W = np.random.normal(0, np.math.sqrt(0.1), (num_of_classes, col, 1))
    B = np.random.normal(0, np.math.sqrt(0.1), (num_of_classes, 1))
    epochs = 70
    G_Train = np.zeros((num_of_classes, epochs + 1))
    ACC_Train = np.zeros((num_of_classes, epochs + 1))
    num_epoch = np.zeros(num_of_classes)
    run = True
    shuff = np.arange(M)
    L_shuff = np.zeros((M, 1))
    X_shuff = np.zeros((M, col, 1))
    desired_acc = 98
    
    for num_class in range(num_of_classes):
        w = W[num_class][:][:]
        b = B[num_class][:]
        epoch = 0
        if dec:
            alpha = dec*alpha
        while(True):
            g = landa*np.dot(np.transpose(w), w)
            acc = 0
            for t in range(M):
                x = Train[t][:][:]
                wx = np.dot(np.transpose(w), x)
                L = TL[num_class][t][:]
                A = L*(wx - b)
                g += (1/M)*(max(0, 1 - A))
                if A > 0:
                    acc += 1
            ACC_Train[num_class][epoch] = (acc/M)*100
            accuracy = ACC_Train[num_class][epoch]
            G_Train[num_class][epoch] = g
            epoch += 1
            run = (epoch < (epochs+1)) and (accuracy < desired_acc)
            if run:
                np.random.shuffle(shuff)
                X_shuff = Train[shuff][:][:]
                L_shuff = TL[num_class][shuff][:]
                w += alpha*((-2*landa)*w)
                for i in range(M):
                    x = X_shuff[i][:][:]
                    L = L_shuff[i][:]
                    wx = np.dot(np.transpose(w), x)
                    A = L*(wx - b)
                    if A < 1:
                        w += alpha*((1/M)*L*x)
                        b = b - (1/M)*L
            else:
                break
        W[num_class][:][:] = w
        B[num_class][:] = b
        num_epoch[num_class] = epoch + 1
    
    vec = np.zeros((num_of_classes, 1))
    pred = np.zeros((num_test, 1))
    #Testing test data
    acc = 0
    for i in range(num_test):
        x = Test_Data[i][:][:]
        for num_class in range(num_of_classes):
            w = W[num_class][:][:]
            b = B[num_class][:]
            wx = np.dot(np.transpose(w), x)
            ww = np.math.sqrt(np.dot(np.transpose(w), w))
            vec[num_class][:] = (wx - b)/ww
        pred[i][:] = np.argmax(vec)
        if int(pred[i][:]) == int(Test_Labels[i][:]):
            acc += 1
    ACC_Test = (acc/num_test)*100
    return W, B, num_epoch, G_Train, ACC_Train, ACC_Test, pred, vec
    

In [10]:
dec = False
landa = 1e-2
alpha = 1e-2
W, B, num_epoch, G_Train, ACC_Train, ACC_Test, pred, vec = steepest_descent(Train, TL, Test_Data, Test_L, Test_Labels, alpha, landa, dec)


In [11]:
print(f"Accuracy using steepest descent optimization = {ACC_Test}")

Accuracy using steepest descent optimization = 76.45041440411546
